In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import json
from typing import List, Dict
import re


class HerkeyJobScraper:
    def __init__(self, headless=True):
        """Initialize the scraper with optional headless mode"""
        self.chrome_options = Options()
        if headless:
            self.chrome_options.add_argument("--headless")
        self.chrome_options.add_argument("--disable-gpu")
        self.chrome_options.add_argument("--window-size=1920,1080")
        self.chrome_options.add_argument("--no-sandbox")
        self.chrome_options.add_argument("--disable-dev-shm-usage")
        
    def scrape_jobs(self, url: str = "https://www.herkey.com/jobs", max_scroll=5, scroll_pause=2) -> List[Dict]:
        """
        Scrape job listings from Herkey
        
        Args:
            url: The URL to scrape
            max_scroll: Maximum number of scroll actions to perform (to load more jobs)
            scroll_pause: Time to pause between scrolls (seconds)
            
        Returns:
            List of job dictionaries containing details of each job
        """
        driver = webdriver.Chrome(options=self.chrome_options)
        
        try:
            # Load the page
            driver.get(url)
            print("Page loaded, waiting for job elements...")
            
            # Wait for job elements to load
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "[data-test-id='job-details']"))
            )
            
            # Scroll down to load more jobs
            for i in range(max_scroll):
                print(f"Scrolling iteration {i+1}/{max_scroll}...")
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(scroll_pause)
            
            # Get all job divs
            job_elements = driver.find_elements(By.CSS_SELECTOR, "[data-test-id='job-details']")
            print(f"Found {len(job_elements)} job listings")
            
            jobs = []
            for job_element in job_elements:
                try:
                    # Extract job details using data-test-id attributes
                    job_title = job_element.find_element(By.CSS_SELECTOR, "[data-test-id='job-title']").text
                    company_name = job_element.find_element(By.CSS_SELECTOR, "[data-test-id='company-name']").text
                    
                    # Get location, work type and experience
                    location_info = job_element.find_element(By.CSS_SELECTOR, "p.capitalize").text
                    
                    # Parse location info - typically in format "City | work type | experience"
                    location_parts = location_info.split('|')
                    location = location_parts[0].strip() if len(location_parts) > 0 else ""
                    work_type = location_parts[1].strip() if len(location_parts) > 1 else ""
                    experience = location_parts[2].strip() if len(location_parts) > 2 else ""
                    
                    # Try to get skills (not all jobs may have this)
                    try:
                        skills_element = job_element.find_element(By.CSS_SELECTOR, "span.capitalize")
                        skills = skills_element.text
                    except:
                        skills = ""
                    
                    # Check if the job has "Easy Apply" option
                    try:
                        apply_button = job_element.find_element(By.CSS_SELECTOR, "[data-test-id='apply-job']")
                        easy_apply = "Easy Apply" in apply_button.text
                    except:
                        easy_apply = False
                    
                    # Get company logo URL if available
                    try:
                        logo_element = job_element.find_element(By.CSS_SELECTOR, "[data-test-id='company-logo'] img")
                        company_logo = logo_element.get_attribute("src")
                    except:
                        company_logo = ""
                    
                    # Create job dictionary
                    job = {
                        "title": job_title,
                        "company": company_name,
                        "location": location,
                        "work_type": work_type,
                        "experience": experience,
                        "skills": skills,
                        "easy_apply": easy_apply,
                        "company_logo": company_logo
                    }
                    
                    jobs.append(job)
                    
                except Exception as e:
                    print(f"Error extracting job details: {e}")
                    continue
            
            return jobs
            
        except TimeoutException:
            print("Timeout waiting for page to load")
            return []
        
        except Exception as e:
            print(f"Error: {e}")
            return []
            
        finally:
            driver.quit()
    
    def save_to_json(self, jobs: List[Dict], filename: str = "herkey_jobs.json"):
        """Save scraped jobs to a JSON file"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(jobs, f, ensure_ascii=False, indent=4)
        print(f"Saved {len(jobs)} jobs to {filename}")
    
    def job_recommendation_data(self, jobs: List[Dict]) -> List[Dict]:
        """
        Process job data to make it suitable for AI recommendation engine
        Extracts and standardizes skills, experience and other relevant fields
        """
        processed_jobs = []
        
        for job in jobs:
            processed_job = job.copy()
            
            # Extract years of experience as a numeric value
            exp_text = job.get("experience", "")
            years_match = re.search(r'(\d+)-(\d+)\s*Yr', exp_text)
            if years_match:
                min_exp = int(years_match.group(1))
                max_exp = int(years_match.group(2))
                processed_job["min_experience"] = min_exp
                processed_job["max_experience"] = max_exp
                processed_job["avg_experience"] = (min_exp + max_exp) / 2
            else:
                processed_job["min_experience"] = None
                processed_job["max_experience"] = None
                processed_job["avg_experience"] = None
            
            # Process skills into a list
            skills_text = job.get("skills", "")
            # Remove the bullet points and '+n' suffix
            skills_text = re.sub(r'\+\d+', '', skills_text)
            skills_list = [skill.strip() for skill in skills_text.split('•') if skill.strip()]
            processed_job["skills_list"] = skills_list
            
            # Standardize work type
            work_type = job.get("work_type", "").lower()
            if "remote" in work_type or "from home" in work_type:
                processed_job["work_mode"] = "remote"
            elif "office" in work_type:
                processed_job["work_mode"] = "in-office"
            elif "hybrid" in work_type:
                processed_job["work_mode"] = "hybrid"
            else:
                processed_job["work_mode"] = "unknown"
            
            processed_jobs.append(processed_job)
        
        return processed_jobs


def recommend_jobs(candidate_profile: Dict, jobs: List[Dict], num_recommendations: int = 5) -> List[Dict]:
    """
    Recommend jobs based on candidate profile
    
    Args:
        candidate_profile: Dictionary containing candidate skills, experience, etc.
        jobs: List of job dictionaries
        num_recommendations: Number of jobs to recommend
        
    Returns:
        List of recommended job dictionaries
    """
    # This is a simple recommendation algorithm as a placeholder
    # In a real implementation, you might use more sophisticated matching or ML
    
    scored_jobs = []
    
    for job in jobs:
        score = 0
        
        # Match skills (biggest factor)
        candidate_skills = [skill.lower() for skill in candidate_profile.get("skills", [])]
        job_skills = [skill.lower() for skill in job.get("skills_list", [])]
        
        for skill in candidate_skills:
            if any(skill in job_skill for job_skill in job_skills):
                score += 10
        
        # Match experience
        candidate_exp = candidate_profile.get("years_of_experience", 0)
        min_exp = job.get("min_experience")
        max_exp = job.get("max_experience")
        
        if min_exp is not None and max_exp is not None:
            if min_exp <= candidate_exp <= max_exp:
                score += 5
            elif candidate_exp < min_exp:
                # Candidate has less experience than required
                score -= (min_exp - candidate_exp) * 2
            else:
                # Candidate is overqualified but might still be interested
                score -= min(5, (candidate_exp - max_exp))
        
        # Match work mode preference
        if candidate_profile.get("preferred_work_mode") == job.get("work_mode"):
            score += 3
        
        # Match location preference
        candidate_locations = candidate_profile.get("preferred_locations", [])
        job_location = job.get("location", "").lower()
        
        if not candidate_locations or any(loc.lower() in job_location for loc in candidate_locations):
            score += 2
        
        scored_jobs.append((score, job))
    
    # Sort by score in descending order and take top recommendations
    scored_jobs.sort(reverse=True, key=lambda x: x[0])
    recommended_jobs = [job for score, job in scored_jobs[:num_recommendations]]
    
    return recommended_jobs


if __name__ == "__main__":
    # Example usage
    scraper = HerkeyJobScraper(headless=False)  # Set to True for headless mode
    jobs = scraper.scrape_jobs(max_scroll=3)
    
    # Save raw job data
    scraper.save_to_json(jobs)
    
    # Process data for recommendation engine
    processed_jobs = scraper.job_recommendation_data(jobs)
    
    # Example candidate profile
    example_candidate = {
        "name": "Jane Doe",
        "years_of_experience": 3,
        "skills": ["React", "JavaScript", "Node.js", "HTML", "CSS"],
        "preferred_work_mode": "remote",
        "preferred_locations": ["Bangalore", "Remote"]
    }
    
    # Get recommendations
    recommendations = recommend_jobs(example_candidate, processed_jobs)
    
    print(f"\nRecommended jobs for {example_candidate['name']}:")
    for i, job in enumerate(recommendations, 1):
        print(f"{i}. {job['title']} at {job['company']} ({job['location']} | {job['work_type']})")
        print(f"   Skills: {job['skills']}")
        print()

Page loaded, waiting for job elements...
Scrolling iteration 1/3...
Scrolling iteration 2/3...
Scrolling iteration 3/3...
Found 60 job listings
Saved 60 jobs to herkey_jobs.json

Recommended jobs for Jane Doe:
1. Software Engineer at Finastra (Pune | Work From Office)
   Skills: C# • Javascript +8

2. Software Engineer at Finastra (Pune | Work From Office)
   Skills: C# • Javascript +8

3. Internship | Work from Home at BTS360 (formerly GetSup... (Bangalore | Work From Home)
   Skills: Customer Service • Sales +1

4. Work from Home Internship at BTS360 (formerly GetSup... (Bangalore | Work From Home)
   Skills: Sales • Client Acquisition

5. English Trainer at FRND App (Any | Work From Home)
   Skills: Teaching English • Communication Skill +2



In [2]:
import requests
from bs4 import BeautifulSoup
from typing import List, Dict
import json
import logging
import re
from datetime import datetime

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class HerkeyEventScraper:
    def __init__(self):
        """Initialize the event scraper"""
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
    
    def _get_page_content(self, url: str) -> str:
        """Get HTML content from a URL"""
        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            return response.text
        except requests.exceptions.RequestException as e:
            logger.error(f"Error fetching URL {url}: {e}")
            return ""
    
    def scrape_events(self, url: str = "https://events.herkey.com/events") -> List[Dict]:
        """Scrape events from HerKey events page."""
        logger.info(f"Scraping events from {url}")
        html_content = self._get_page_content(url)
        if not html_content:
            return []
            
        soup = BeautifulSoup(html_content, 'html.parser')
        events = []
        
        # Extract event listings based on the provided HTML structure
        event_cards = soup.select(".event-details-card")
        
        for card in event_cards:
            try:
                event = {}
                
                # Extract event details based on the actual HTML structure
                title_elem = card.select_one(".card-heading")
                date_elem = card.select_one(".card-body-data:has(img[src*='calendar'])")
                time_elem = card.select_one(".card-body-data:has(img[src*='clock'])")
                location_elem = card.select_one(".card-body-data:has(img[src*='placeholder'])")
                event_type_elem = card.select_one("span.mr-1, .card-body-data:has(i.fa-bullseye)")
                category_elem = card.select_one(".card-body-data:has(img[src*='tag'])")
                
                if title_elem:
                    title_text = title_elem.get_text(strip=True)
                    # Remove any icon text from the title
                    if title_text:
                        event["title"] = title_text.split("Featured")[0].strip()
                
                if date_elem:
                    date_text = date_elem.get_text(strip=True)
                    if date_text:
                        event["date"] = date_text
                
                if time_elem:
                    time_text = time_elem.get_text(strip=True)
                    if time_text:
                        event["time"] = time_text
                
                if location_elem:
                    location_text = location_elem.get_text(strip=True)
                    if location_text:
                        event["location"] = location_text
                
                if event_type_elem:
                    event_type = event_type_elem.get_text(strip=True)
                    if event_type and "Online" in event_type or "Offline" in event_type:
                        event["event_type"] = event_type
                
                if category_elem:
                    # Parse categories (they're in a special format)
                    category_links = category_elem.select("a")
                    if category_links:
                        event["categories"] = [link.text.strip() for link in category_links]
                
                # Get the registration button type
                register_btn = card.select_one(".register")
                if register_btn:
                    event["registration_status"] = "Open"
                    event["registration_text"] = register_btn.text.strip()
                
                # Check if it's a paid event
                price_elem = card.select_one(".card-body-data:contains('₹')")
                if price_elem:
                    event["price"] = price_elem.text.strip()
                else:
                    event["price"] = "Free"
                
                # Get event ID and URL
                event_id_span = card.select_one("span[id]")
                if event_id_span and event_id_span.get('id'):
                    event_id = event_id_span.get('id')
                    if "title" in event:
                        event["url"] = f"{url}/{event['title'].lower().replace(' ', '-')}/{event_id}"
                
                # Check if it's a featured event
                featured_elem = card.select_one("img[src*='Featured'], img[width='22'][height='22']")
                if featured_elem:
                    event["featured"] = True
                
                if event:  # Only add if we found some data
                    events.append(event)
            except Exception as e:
                logger.error(f"Error extracting event data: {e}")
                
        logger.info(f"Scraped {len(events)} events")
        return events
    
    def save_to_json(self, events: List[Dict], filename: str = "herkey_events.json"):
        """Save scraped events to a JSON file"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(events, f, ensure_ascii=False, indent=4)
        logger.info(f"Saved {len(events)} events to {filename}")
    
    def process_events_for_recommendation(self, events: List[Dict]) -> List[Dict]:
        """
        Process event data to make it suitable for AI recommendation engine
        Extracts and standardizes categories, dates and other relevant fields
        """
        processed_events = []
        
        for event in events:
            processed_event = event.copy()
            
            # Process date to datetime object if possible
            date_str = event.get("date", "")
            try:
                # Example format: "25 Apr 2023" - adjust pattern as needed
                date_obj = datetime.strptime(date_str, "%d %b %Y")
                processed_event["datetime_obj"] = date_obj
                processed_event["is_upcoming"] = date_obj > datetime.now()
            except (ValueError, TypeError):
                # If we can't parse the date, keep it as is
                processed_event["is_upcoming"] = True  # Assume upcoming by default
            
            # Standardize event type
            event_type = event.get("event_type", "").lower()
            if "online" in event_type:
                processed_event["mode"] = "online"
            elif "offline" in event_type or "in-person" in event_type:
                processed_event["mode"] = "offline"
            else:
                processed_event["mode"] = "unknown"
            
            # Standardize categories
            categories = event.get("categories", [])
            processed_event["categories_lower"] = [cat.lower() for cat in categories]
            
            # Determine if event is free
            price = event.get("price", "")
            processed_event["is_free"] = price == "Free" or "free" in price.lower()
            
            processed_events.append(processed_event)
        
        return processed_events


def recommend_events(candidate_profile: Dict, events: List[Dict], num_recommendations: int = 5) -> List[Dict]:
    """
    Recommend events based on candidate profile
    
    Args:
        candidate_profile: Dictionary containing candidate interests, career stage, etc.
        events: List of event dictionaries
        num_recommendations: Number of events to recommend
        
    Returns:
        List of recommended event dictionaries
    """
    # Process events for recommendation if they haven't been processed
    if events and "categories_lower" not in events[0]:
        scraper = HerkeyEventScraper()
        events = scraper.process_events_for_recommendation(events)
    
    scored_events = []
    
    # Get candidate preferences
    interests = [interest.lower() for interest in candidate_profile.get("interests", [])]
    preferred_mode = candidate_profile.get("preferred_event_mode", "").lower()
    preferred_locations = [loc.lower() for loc in candidate_profile.get("preferred_locations", [])]
    career_stage = candidate_profile.get("career_stage", "").lower()
    
    # Only consider upcoming events
    upcoming_events = [event for event in events if event.get("is_upcoming", True)]
    
    for event in upcoming_events:
        score = 0
        
        # Match interests with event categories
        event_categories = event.get("categories_lower", [])
        for interest in interests:
            if any(interest in category for category in event_categories):
                score += 10
        
        # Match event mode preference (online/offline)
        event_mode = event.get("mode", "unknown")
        if preferred_mode and event_mode == preferred_mode:
            score += 5
        
        # Match location preference if it's an offline event
        if event_mode == "offline":
            location = event.get("location", "").lower()
            if any(loc in location for loc in preferred_locations):
                score += 5
        
        # Free events might be more appealing
        if event.get("is_free", False):
            score += 2
        
        # Featured events might be more relevant/important
        if event.get("featured", False):
            score += 3
        
        # Career stage specific events
        event_title = event.get("title", "").lower()
        if career_stage in event_title or any(career_stage in cat for cat in event_categories):
            score += 4
        
        scored_events.append((score, event))
    
    # Sort by score in descending order and take top recommendations
    scored_events.sort(reverse=True, key=lambda x: x[0])
    recommended_events = [event for score, event in scored_events[:num_recommendations]]
    
    return recommended_events


if __name__ == "__main__":
    # Example usage
    scraper = HerkeyEventScraper()
    events = scraper.scrape_events()
    
    # Save raw event data
    scraper.save_to_json(events)
    
    # Process data for recommendation engine
    processed_events = scraper.process_events_for_recommendation(events)
    
    # Example candidate profile
    example_candidate = {
        "name": "Jane Doe",
        "interests": ["Technology", "Career Development", "Leadership", "AI"],
        "preferred_event_mode": "online",
        "preferred_locations": ["Bangalore", "Mumbai", "Delhi"],
        "career_stage": "mid-level"
    }
    
    # Get recommendations
    recommendations = recommend_events(example_candidate, processed_events)
    
    print(f"\nRecommended events for {example_candidate['name']}:")
    for i, event in enumerate(recommendations, 1):
        print(f"{i}. {event['title']}")
        print(f"   Date: {event['date']} | Time: {event.get('time', 'N/A')}")
        print(f"   Type: {event.get('event_type', 'N/A')} | Price: {event.get('price', 'N/A')}")
        print(f"   Categories: {', '.join(event.get('categories', []))}")
        print()

2025-04-25 08:16:29,537 - __main__ - INFO - Scraping events from https://events.herkey.com/events
c:\Users\knand\OneDrive\Desktop\AshaAIBot\asha\Lib\site-packages\soupsieve\css_parser.py:876: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028
2025-04-25 08:16:31,204 - __main__ - INFO - Scraped 8 events
2025-04-25 08:16:31,206 - __main__ - INFO - Saved 8 events to herkey_events.json



Recommended events for Jane Doe:
1. Advance Your Career with a Premier Executive MBA from Great Lakes
   Date: 16th Apr, 2025 to 30th Apr, 2025 | Time: 10:00am to 6:00pm
   Type: Online | Price: Free
   Categories: Career Development, event

2. SkillReBoot program: Restart Your Career Journey
   Date: 21st Apr, 2025 to 20th May, 2025 | Time: 6:00pm to 11:00pm
   Type: Online | Price: Free
   Categories: Career Development

3. #STEMTheBias Scholarship - Avail Scholarships up to Rs. 80,000
   Date: 3rd Mar, 2025 to 25th Mar, 2025 | Time: 10:00am to 11:59pm
   Type: Online | Price: Free
   Categories: Career Development

4. LeadHERs in Tech: Tech Meets Talent , Innovation Meets Inspiration
   Date: 21st Mar, 2025 | Time: 8:30am to 4:00pm
   Type: Offline | Price: Free
   Categories: Career Development, Women In Tech, networking

5. herShakti
   Date: 8th Mar, 2025 to 10th Apr, 2025 | Time: 12:00am to 11:00pm
   Type: Online | Price: Free
   Categories: JobsForHer foundation



In [3]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import json
from typing import List, Dict
import logging
import re

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class HerkeyGroupScraper:
    def __init__(self, headless=True):
        """Initialize the group scraper with optional headless mode"""
        self.chrome_options = Options()
        if headless:
            self.chrome_options.add_argument("--headless")
        self.chrome_options.add_argument("--disable-gpu")
        self.chrome_options.add_argument("--window-size=1920,1080")
        self.chrome_options.add_argument("--no-sandbox")
        self.chrome_options.add_argument("--disable-dev-shm-usage")
    
    def scrape_groups(self, url: str = "https://www.herkey.com/groups", max_scroll=5, scroll_pause=2) -> List[Dict]:
        """
        Scrape group listings from Herkey
        
        Args:
            url: The URL to scrape
            max_scroll: Maximum number of scroll actions to perform (to load more groups)
            scroll_pause: Time to pause between scrolls (seconds)
            
        Returns:
            List of group dictionaries containing details of each group
        """
        driver = webdriver.Chrome(options=self.chrome_options)
        
        try:
            # Load the page
            driver.get(url)
            logger.info("Page loaded, waiting for group elements...")
            
            # Wait for group elements to load
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "[data-test-id='featured-group']"))
            )
            
            # Scroll down to load more groups
            for i in range(max_scroll):
                logger.info(f"Scrolling iteration {i+1}/{max_scroll}...")
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(scroll_pause)
            
            # Get page source after all content is loaded
            page_content = driver.page_source
            soup = BeautifulSoup(page_content, 'html.parser')
            
            # Find all group container elements
            group_elements = soup.select("div.MuiGrid-container.css-1d3bbye")
            logger.info(f"Found {len(group_elements)} group elements")
            
            groups = []
            for group_element in group_elements:
                try:
                    group = {}
                    
                    # Extract group name
                    name_elem = group_element.select_one("[data-test-id='group-name']")
                    if name_elem:
                        group["name"] = name_elem.text.strip()
                    
                    # Extract group type (private/public)
                    type_elem = group_element.select_one("[data-test-id='group-type']")
                    if type_elem:
                        group["type"] = type_elem.text.strip().lower()
                    
                    # Extract member count
                    members_elem = group_element.select_one("[data-test-id='group-members-count']")
                    if members_elem:
                        members_text = members_elem.text.strip()
                        # Extract number from text like "43 Members"
                        members_match = re.search(r'(\d+)\s+Members?', members_text)
                        if members_match:
                            group["member_count"] = int(members_match.group(1))
                    
                    # Check if it's a featured group
                    featured_elem = group_element.select_one("[data-test-id='featured-icon']")
                    group["featured"] = featured_elem is not None
                    
                    # Extract group icon URL
                    icon_elem = group_element.select_one("[data-test-id='group-icon']")
                    if icon_elem and icon_elem.has_attr('src'):
                        group["icon_url"] = icon_elem['src']
                    
                    # Extract banner image if available
                    banner_elem = group_element.select_one(".css-12c20jy img")
                    if banner_elem and banner_elem.has_attr('src'):
                        group["banner_url"] = banner_elem['src']
                    
                    # Extract group category/topic if available
                    # Note: This might not be directly visible in the HTML snippet provided
                    # We'll use a more generic approach to try and find it
                    category_elem = group_element.select_one(".MuiTypography-root.capitalize")
                    if category_elem:
                        group["category"] = category_elem.text.strip()
                    
                    # Extract join button status
                    join_btn = group_element.select_one("[data-test-id='join-btn'] button")
                    if join_btn:
                        group["joinable"] = True
                        group["join_button_text"] = join_btn.text.strip()
                    else:
                        group["joinable"] = False
                    
                    if "name" in group:  # Only add if we have at least a name
                        groups.append(group)
                        
                except Exception as e:
                    logger.error(f"Error extracting group data: {e}")
            
            return groups
            
        except TimeoutException:
            logger.error("Timeout waiting for page to load")
            return []
        
        except Exception as e:
            logger.error(f"Error: {e}")
            return []
            
        finally:
            driver.quit()
    
    def save_to_json(self, groups: List[Dict], filename: str = "herkey_groups.json"):
        """Save scraped groups to a JSON file"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(groups, f, ensure_ascii=False, indent=4)
        logger.info(f"Saved {len(groups)} groups to {filename}")
    
    def process_groups_for_recommendation(self, groups: List[Dict]) -> List[Dict]:
        """
        Process group data to make it suitable for AI recommendation engine
        Extracts and standardizes relevant fields
        """
        processed_groups = []
        
        for group in groups:
            processed_group = group.copy()
            
            # Extract topics/skills from group name where possible
            name = group.get("name", "").lower()
            
            # Extract potential skills/technologies
            tech_keywords = [
                "javascript", "python", "java", "c++", "ruby", "php", "golang", "react", 
                "angular", "vue", "node", "express", "django", "flask", "spring", "bootstrap",
                "html", "css", "sql", "nosql", "mongodb", "mysql", "postgresql", "oracle",
                "aws", "azure", "gcp", "cloud", "devops", "data science", "machine learning",
                "ai", "artificial intelligence", "blockchain", "iot", "mobile", "android",
                "ios", "swift", "kotlin", "flutter", "react native", "full stack", "frontend",
                "backend", "ui", "ux", "design", "product", "agile", "scrum", "kanban",
                "mern", "mean", "lamp", "microservices", "docker", "kubernetes", "jenkins",
                "ci/cd", "testing", "qa", "security", "cyber security", "data engineering",
                "big data", "hadoop", "spark", "tableau", "power bi", "data visualization"
            ]
            
            # Look for tech keywords in the group name
            found_keywords = [keyword for keyword in tech_keywords if keyword in name]
            processed_group["tech_keywords"] = found_keywords
            
            # Determine if group is for beginners, intermediate, or advanced
            level_indicators = {
                "beginner": ["beginner", "basic", "fundamental", "101", "intro", "start"],
                "intermediate": ["intermediate", "mid-level"],
                "advanced": ["advanced", "expert", "professional", "master", "senior"]
            }
            
            for level, indicators in level_indicators.items():
                if any(indicator in name for indicator in indicators):
                    processed_group["level"] = level
                    break
            else:
                processed_group["level"] = "all"  # Default if no level is detected
            
            # Add group category based on name (simplified)
            if any(term in name for term in ["developer", "coding", "programming", "engineer", "mern", "stack"]):
                processed_group["category"] = "development"
            elif any(term in name for term in ["design", "ui", "ux", "user experience"]):
                processed_group["category"] = "design"
            elif any(term in name for term in ["data", "analytics", "science", "machine learning", "ai"]):
                processed_group["category"] = "data_science"
            elif any(term in name for term in ["management", "leader", "agile", "scrum", "product"]):
                processed_group["category"] = "management"
            elif any(term in name for term in ["testing", "qa", "quality"]):
                processed_group["category"] = "testing"
            elif any(term in name for term in ["devops", "cloud", "aws", "azure", "gcp"]):
                processed_group["category"] = "devops"
            else:
                processed_group["category"] = "other"
            
            processed_groups.append(processed_group)
        
        return processed_groups


def recommend_groups(candidate_profile: Dict, groups: List[Dict], num_recommendations: int = 5) -> List[Dict]:
    """
    Recommend groups based on candidate profile
    
    Args:
        candidate_profile: Dictionary containing candidate skills, experience level, etc.
        groups: List of group dictionaries
        num_recommendations: Number of groups to recommend
        
    Returns:
        List of recommended group dictionaries
    """
    # Process groups for recommendation if they haven't been processed
    if groups and "tech_keywords" not in groups[0]:
        scraper = HerkeyGroupScraper()
        groups = scraper.process_groups_for_recommendation(groups)
    
    scored_groups = []
    
    # Get candidate preferences
    skills = [skill.lower() for skill in candidate_profile.get("skills", [])]
    experience_level = candidate_profile.get("experience_level", "").lower()
    
    # Map experience level to group levels
    level_mapping = {
        "entry": ["beginner", "all"],
        "junior": ["beginner", "intermediate", "all"],
        "mid-level": ["intermediate", "all"],
        "senior": ["intermediate", "advanced", "all"],
        "lead": ["advanced", "all"]
    }
    
    suitable_levels = level_mapping.get(experience_level, ["all"])
    
    for group in groups:
        score = 0
        
        # Match skills with group tech keywords
        group_keywords = group.get("tech_keywords", [])
        for skill in skills:
            if any(skill in keyword for keyword in group_keywords):
                score += 10
            # Also check if skill appears in the group name
            if skill in group.get("name", "").lower():
                score += 5
        
        # Match experience level with group level
        group_level = group.get("level", "all")
        if group_level in suitable_levels:
            score += 5
        
        # Featured groups might be more relevant/important
        if group.get("featured", False):
            score += 3
        
        # Groups with more members might be more valuable
        member_count = group.get("member_count", 0)
        if member_count > 100:
            score += 3
        elif member_count > 50:
            score += 2
        elif member_count > 10:
            score += 1
        
        # Category matching
        candidate_interests = [interest.lower() for interest in candidate_profile.get("interests", [])]
        group_category = group.get("category", "").lower()
        
        if any(interest in group_category for interest in candidate_interests):
            score += 5
        
        scored_groups.append((score, group))
    
    # Sort by score in descending order and take top recommendations
    scored_groups.sort(reverse=True, key=lambda x: x[0])
    recommended_groups = [group for score, group in scored_groups[:num_recommendations]]
    
    return recommended_groups


if __name__ == "__main__":
    # Example usage
    scraper = HerkeyGroupScraper(headless=False)  # Set to True for headless mode
    groups = scraper.scrape_groups(max_scroll=3)
    
    # Save raw group data
    scraper.save_to_json(groups)
    
    # Process data for recommendation engine
    processed_groups = scraper.process_groups_for_recommendation(groups)
    
    # Example candidate profile
    example_candidate = {
        "name": "Jane Doe",
        "skills": ["React", "JavaScript", "Node.js", "MongoDB", "Express"],
        "experience_level": "mid-level",
        "interests": ["development", "web development", "frontend", "backend"]
    }
    
    # Get recommendations
    recommendations = recommend_groups(example_candidate, processed_groups)
    
    print(f"\nRecommended groups for {example_candidate['name']}:")
    for i, group in enumerate(recommendations, 1):
        print(f"{i}. {group['name']}")
        print(f"   Type: {group.get('type', 'N/A')} | Members: {group.get('member_count', 'N/A')}")
        print(f"   Category: {group.get('category', 'N/A')} | Level: {group.get('level', 'N/A')}")
        print(f"   Featured: {'Yes' if group.get('featured', False) else 'No'}")
        print()

2025-04-25 08:20:11,457 - __main__ - INFO - Page loaded, waiting for group elements...
2025-04-25 08:20:12,870 - __main__ - INFO - Scrolling iteration 1/3...
2025-04-25 08:20:15,220 - __main__ - INFO - Scrolling iteration 2/3...
2025-04-25 08:20:17,434 - __main__ - INFO - Scrolling iteration 3/3...
2025-04-25 08:20:19,658 - __main__ - INFO - Found 104 group elements
2025-04-25 08:20:22,126 - __main__ - INFO - Saved 103 groups to herkey_groups.json



Recommended groups for Jane Doe:
1. Women Engineers
   Type: public | Members: 485
   Category: development | Level: all
   Featured: No

2. Full Stack (MERN) Developer Program
   Type: private | Members: N/A
   Category: development | Level: all
   Featured: Yes

3. Ambassadors Club
   Type: private | Members: 4129
   Category: other | Level: all
   Featured: Yes

4. Women on a Career Break
   Type: public | Members: 36445
   Category: other | Level: all
   Featured: Yes

5. Women's March
   Type: public | Members: 6066
   Category: other | Level: all
   Featured: Yes



In [5]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import json
import logging
import re
from datetime import datetime
from typing import List, Dict

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class HerkeySessionScraper:
    def __init__(self, headless=True):
        """Initialize the session scraper with optional headless mode"""
        self.chrome_options = Options()
        if headless:
            self.chrome_options.add_argument("--headless")
        self.chrome_options.add_argument("--disable-gpu")
        self.chrome_options.add_argument("--window-size=1920,1080")
        self.chrome_options.add_argument("--no-sandbox")
        self.chrome_options.add_argument("--disable-dev-shm-usage")
    
    def scrape_sessions(self, url: str = "https://www.herkey.com/sessions", max_scroll=5, scroll_pause=2) -> List[Dict]:
        """
        Scrape session listings from Herkey
        
        Args:
            url: The URL to scrape
            max_scroll: Maximum number of scroll actions to perform (to load more sessions)
            scroll_pause: Time to pause between scrolls (seconds)
            
        Returns:
            List of session dictionaries containing details of each session
        """
        driver = webdriver.Chrome(options=self.chrome_options)
        
        try:
            # Load the page
            driver.get(url)
            logger.info("Page loaded, waiting for session elements...")
            
            # Wait for session elements to load
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "[datatestid='session-card']"))
            )
            
            # Scroll down to load more sessions
            for i in range(max_scroll):
                logger.info(f"Scrolling iteration {i+1}/{max_scroll}...")
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(scroll_pause)
            
            # Get all session cards
            session_elements = driver.find_elements(By.CSS_SELECTOR, "[datatestid='session-card'], [data-test-id='session-card']")
            logger.info(f"Found {len(session_elements)} session listings")
            
            sessions = []
            for session_element in session_elements:
                try:
                    session = {}
                    
                    # Extract session time, date
                    try:
                        time_data = session_element.find_element(By.CSS_SELECTOR, "[data-test-id='time-data']").text
                        if time_data:
                            # Parse date and time (format: "24 Apr 25 | 10:30 AM")
                            date_time_parts = time_data.split('|')
                            if len(date_time_parts) > 0:
                                session["date"] = date_time_parts[0].strip()
                            if len(date_time_parts) > 1:
                                session["time"] = date_time_parts[1].strip()
                    except:
                        logger.debug("Could not extract time data")
                    
                    # Extract session title
                    try:
                        discussion_subject = session_element.find_element(By.CSS_SELECTOR, "[data-test-id='discussion-subject'] p").text
                        if discussion_subject:
                            session["title"] = discussion_subject
                    except:
                        logger.debug("Could not extract session title")
                    
                    # Extract host info
                    try:
                        host_element = session_element.find_element(By.CSS_SELECTOR, "[data-test-id='nav-to-user-profile'] h6")
                        if host_element:
                            session["host"] = host_element.text
                    except:
                        logger.debug("Could not extract host name")
                    
                    # Extract host headline/role if available
                    try:
                        headline_element = session_element.find_element(By.CSS_SELECTOR, "[data-test-id='headline']")
                        if headline_element and headline_element.text.strip():
                            session["host_headline"] = headline_element.text
                    except:
                        logger.debug("Could not extract host headline")
                    
                    # Get host stage/level
                    try:
                        stage_element = session_element.find_element(By.CSS_SELECTOR, "[data-test-id='profile-stage'] span")
                        if stage_element:
                            session["host_stage"] = stage_element.text
                    except:
                        logger.debug("Could not extract host stage")
                    
                    # Get participant count
                    try:
                        participant_element = session_element.find_element(By.CSS_SELECTOR, "[data-test-id='profile-pic']")
                        participant_text = participant_element.text
                        if participant_text and participant_text.startswith("+"):
                            session["participant_count"] = int(participant_text.replace("+", "").strip())
                    except:
                        logger.debug("Could not extract participant count")
                    
                    # Get session type (past/upcoming)
                    try:
                        session_type = session_element.get_attribute("data-sessiontype")
                        if session_type:
                            session["session_type"] = session_type
                    except:
                        logger.debug("Could not extract session type")
                    
                    # Get session ID
                    try:
                        session_id = session_element.get_attribute("data-id")
                        if session_id:
                            session["id"] = session_id
                            # Construct URL
                            if "title" in session:
                                slug = session["title"].lower().replace(" ", "-")
                                session["url"] = f"{url}/{slug}/{session_id}"
                    except:
                        logger.debug("Could not extract session ID")
                    
                    # Check if it's a video session
                    try:
                        video_element = session_element.find_element(By.CSS_SELECTOR, "div[style*='youtube.com']")
                        if video_element:
                            session["is_video"] = True
                            video_url = video_element.get_attribute("style")
                            # Extract YouTube video ID
                            youtube_match = re.search(r'youtube.com/vi/([^/]+)/', video_url)
                            if youtube_match:
                                session["youtube_id"] = youtube_match.group(1)
                    except:
                        session["is_video"] = False
                    
                    # Get session status/action button text
                    try:
                        status_button = session_element.find_element(By.CSS_SELECTOR, "[data-test-id='discussion-status-button'] button p")
                        if status_button:
                            session["action"] = status_button.text.strip()
                    except:
                        logger.debug("Could not extract action button text")
                    
                    if session:  # Only add if we found some data
                        sessions.append(session)
                except Exception as e:
                    logger.error(f"Error extracting session data: {e}")
                    continue
            
            return sessions
            
        except TimeoutException:
            logger.error("Timeout waiting for page to load")
            return []
        
        except Exception as e:
            logger.error(f"Error: {e}")
            return []
            
        finally:
            driver.quit()
    
    def save_to_json(self, sessions: List[Dict], filename: str = "herkey_sessions.json"):
        """Save scraped sessions to a JSON file"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(sessions, f, ensure_ascii=False, indent=4)
        logger.info(f"Saved {len(sessions)} sessions to {filename}")
    
    def process_sessions_for_recommendation(self, sessions: List[Dict]) -> List[Dict]:
        """
        Process session data to make it suitable for AI recommendation engine
        Extracts and standardizes dates, topics and other relevant fields
        """
        processed_sessions = []
        
        for session in sessions:
            processed_session = session.copy()
            
            # Process date to datetime object if possible
            date_str = session.get("date", "")
            time_str = session.get("time", "")
            datetime_str = f"{date_str} {time_str}".strip()

            # In the process_sessions_for_recommendation method
            try:
                # Try various date formats
                date_obj = None  # Initialize date_obj to avoid the UnboundLocalError
                
                if len(date_str.split()) == 3:  # Format: "24 Apr 25"
                    date_obj = datetime.strptime(datetime_str, "%d %b %y %I:%M %p")
                else:
                    # Try other formats if the first one fails
                    date_formats = [
                        "%d %b %Y %I:%M %p",  # 24 Apr 2025 10:30 AM
                        "%d %B %Y %I:%M %p",  # 24 April 2025 10:30 AM
                        "%b %d, %Y %I:%M %p"  # Apr 24, 2025 10:30 AM
                    ]
                    
                    for date_format in date_formats:
                        try:
                            date_obj = datetime.strptime(datetime_str, date_format)
                            break
                        except ValueError:
                            continue
                
                # Check if date_obj was successfully set
                if date_obj:
                    processed_session["datetime_obj"] = date_obj
                    processed_session["is_upcoming"] = date_obj > datetime.now()
                else:
                    # Handle case where no format worked
                    raise ValueError("Could not parse date string")
                    
            except (ValueError, TypeError):
                # If we can't parse the date, determine upcoming from session_type
                if session.get("session_type") == "upcoming":
                    processed_session["is_upcoming"] = True
                else:
                    processed_session["is_upcoming"] = False
            
            # Extract topics from title
            title = session.get("title", "").lower()
            topics = []
            
            # Common tech topics to check in the title
            tech_topics = [
                "python", "javascript", "react", "angular", "vue", "node", "java", "c++", "c#", 
                "php", "ruby", "golang", "rust", "swift", "kotlin", "sql", "nosql", "mongodb",
                "database", "cloud", "aws", "azure", "gcp", "docker", "kubernetes", "devops",
                "ai", "machine learning", "data science", "deep learning", "nlp", "computer vision",
                "blockchain", "iot", "mobile", "web", "frontend", "backend", "fullstack", "ui", "ux",
                "testing", "qa", "security", "agile", "scrum", "kanban", "product management",
                "power bi", "tableau", "excel", "analytics", "big data", "hadoop", "spark", 
                "cybersecurity", "networking", "seo", "digital marketing", "dax"
            ]
            
            for topic in tech_topics:
                if topic in title:
                    topics.append(topic)
            
            processed_session["extracted_topics"] = topics
            
            # Determine if session is technical or soft-skills
            tech_indicators = ["programming", "code", "developer", "software", "tech", "data", 
                               "engineering", "algorithm", "system", "database", "cloud", "devops"]
            
            softskill_indicators = ["career", "leadership", "management", "communication",
                                   "soft skill", "interview", "resume", "cv", "personal",
                                   "growth", "mindset", "wellbeing", "mental health"]
            
            tech_score = sum(1 for indicator in tech_indicators if indicator in title)
            softskill_score = sum(1 for indicator in softskill_indicators if indicator in title)
            
            if tech_score > softskill_score:
                processed_session["session_category"] = "technical"
            elif softskill_score > tech_score:
                processed_session["session_category"] = "soft skills"
            else:
                processed_session["session_category"] = "general"
            
            # Determine experience level based on title
            if any(x in title for x in ["beginner", "basic", "introduction", "101", "fundamentals"]):
                processed_session["experience_level"] = "beginner"
            elif any(x in title for x in ["advanced", "expert", "mastery", "professional"]):
                processed_session["experience_level"] = "advanced"
            elif any(x in title for x in ["intermediate", "part 2", "level 2"]):
                processed_session["experience_level"] = "intermediate"
            else:
                processed_session["experience_level"] = "all levels"
            
            processed_sessions.append(processed_session)
        
        return processed_sessions


def recommend_sessions(candidate_profile: Dict, sessions: List[Dict], num_recommendations: int = 5) -> List[Dict]:
    """
    Recommend sessions based on candidate profile
    
    Args:
        candidate_profile: Dictionary containing candidate interests, experience level, etc.
        sessions: List of session dictionaries
        num_recommendations: Number of sessions to recommend
        
    Returns:
        List of recommended session dictionaries
    """
    # Process sessions for recommendation if they haven't been processed
    if sessions and "extracted_topics" not in sessions[0]:
        scraper = HerkeySessionScraper()
        sessions = scraper.process_sessions_for_recommendation(sessions)
    
    scored_sessions = []
    
    # Get candidate preferences
    interests = [interest.lower() for interest in candidate_profile.get("interests", [])]
    skills = [skill.lower() for skill in candidate_profile.get("skills", [])]
    career_stage = candidate_profile.get("career_stage", "").lower()
    experience_level = candidate_profile.get("experience_level", "intermediate").lower()
    career_goals = [goal.lower() for goal in candidate_profile.get("career_goals", [])]
    
    # Combine interests and skills for broader matching
    all_interests = interests + skills
    
    # Only consider upcoming sessions by default unless specified otherwise
    if candidate_profile.get("include_past_sessions", False):
        filtered_sessions = sessions
    else:
        filtered_sessions = [session for session in sessions if session.get("is_upcoming", True)]
    
    for session in filtered_sessions:
        score = 0
        
        # Match extracted topics with interests and skills
        for topic in session.get("extracted_topics", []):
            if any(interest in topic or topic in interest for interest in all_interests):
                score += 10
        
        # Match session title keywords with interests and skills
        title = session.get("title", "").lower()
        for interest in all_interests:
            if interest in title:
                score += 5
        
        # Match experience level
        session_level = session.get("experience_level", "all levels")
        if session_level == "all levels" or session_level == experience_level:
            score += 3
        elif (session_level == "beginner" and experience_level in ["intermediate", "advanced"]) or \
             (session_level == "intermediate" and experience_level == "advanced"):
            # Session might be too basic
            score -= 2
        elif (session_level == "advanced" and experience_level == "beginner"):
            # Session might be too advanced
            score -= 3
        
        # Prefer upcoming sessions over past ones
        if session.get("is_upcoming", True):
            score += 4
        
        # Match session category with career goals
        session_category = session.get("session_category", "general")
        if any(goal in session_category for goal in career_goals) or \
           any(session_category in goal for goal in career_goals):
            score += 3
        
        # Factor in popularity (participant count)
        participant_count = session.get("participant_count", 0)
        if participant_count > 50:
            score += 3
        elif participant_count > 20:
            score += 2
        elif participant_count > 5:
            score += 1
        
        # Video sessions (past recordings) might be immediately available
        if session.get("is_video", False):
            score += 2
        
        scored_sessions.append((score, session))
    
    # Sort by score in descending order and take top recommendations
    scored_sessions.sort(reverse=True, key=lambda x: x[0])
    recommended_sessions = [session for score, session in scored_sessions[:num_recommendations]]
    
    return recommended_sessions


if __name__ == "__main__":
    # Example usage
    scraper = HerkeySessionScraper(headless=False)  # Set to True for headless mode
    sessions = scraper.scrape_sessions(max_scroll=3)
    
    # Save raw session data
    scraper.save_to_json(sessions)
    
    # Process data for recommendation engine
    processed_sessions = scraper.process_sessions_for_recommendation(sessions)
    
    # Example candidate profile
    example_candidate = {
        "name": "John Doe",
        "interests": ["Data Analysis", "Machine Learning", "Power BI", "Python"],
        "skills": ["SQL", "Excel", "Python", "Data Visualization"],
        "career_stage": "mid-level",
        "experience_level": "intermediate",
        "career_goals": ["technical growth", "data science"],
        "include_past_sessions": True  # Include past sessions in recommendations
    }
    
    # Get recommendations
    recommendations = recommend_sessions(example_candidate, processed_sessions)
    
    print(f"\nRecommended sessions for {example_candidate['name']}:")
    for i, session in enumerate(recommendations, 1):
        print(f"{i}. {session['title']}")
        print(f"   Date: {session.get('date', 'N/A')} | Time: {session.get('time', 'N/A')}")
        print(f"   Host: {session.get('host', 'N/A')}")
        print(f"   Type: {session.get('session_type', 'N/A')} | Category: {session.get('session_category', 'N/A')}")
        print(f"   Topics: {', '.join(session.get('extracted_topics', []))}")
        print()

2025-04-25 08:36:58,210 - __main__ - INFO - Page loaded, waiting for session elements...
2025-04-25 08:36:58,229 - __main__ - INFO - Scrolling iteration 1/3...
2025-04-25 08:37:00,240 - __main__ - INFO - Scrolling iteration 2/3...
2025-04-25 08:37:02,252 - __main__ - INFO - Scrolling iteration 3/3...
2025-04-25 08:37:04,284 - __main__ - INFO - Found 29 session listings
2025-04-25 08:37:12,997 - __main__ - INFO - Saved 29 sessions to herkey_sessions.json



Recommended sessions for John Doe:
1. Power BI Masterclass : Calculating Measures with DAX (Part 1)
   Date: Today | Time: 10:30 AM
   Host: Swati Agarwal
   Type: upcoming | Category: general
   Topics: power bi, dax

2. AccelHERate Delhi 2025 | The Future of Hiring: Merging Tech & Human Touch to Unlock Women Talent
   Date: Today | Time: 10:00 AM
   Host: Herkey Events
   Type: upcoming | Category: technical
   Topics: 

3. AccelHERate Pune 2025 | Hiring Reimagined: Balancing Tech & Human Connection to Unlock Women Talent
   Date: 16 May 25 | Time: 10:00 AM
   Host: Herkey Events
   Type: upcoming | Category: technical
   Topics: 

4. Paroma Chatterjee , CEO, Revolut India on transforming the Indian Fintech space.
   Date: Today | Time: 3:00 PM
   Host: Khushboo Ramnane
   Type: upcoming | Category: technical
   Topics: 

5. Opportunity to Network With Employers And Industry Leaders | HerRising 2025 | Bangalore
   Date: 19 Jun 25 | Time: 10:00 AM
   Host: Herkey Events
   Type: upco